In [1]:
# 관련 패키지 임포트
import os
from llama_index.core import Document
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from pprint import pprint
import nest_asyncio

nest_asyncio.apply()

# 활용 LLM API 정보 설정
os.environ["OPENAI_API_KEY"] = ''

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
#Settings.chunk_size = 256

In [2]:
documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
documents

In [ ]:
len(documents)

In [3]:
query_engine = index.as_query_engine()

In [4]:
response = query_engine.query('공무원으로 일하다가 미국인이 되기로 결심했어. 내 연금은 어떻게 되는거야?')

In [5]:
pprint(response.response)

('공무원으로 일하다가 미국인이 되기로 결심한 경우, 연금에 대한 구체적인 규정은 해당 법률에 따라 다를 수 있습니다. 일반적으로, 공무원 '
 '연금은 퇴직 후 지급되며, 퇴직 사유에 따라 지급이 결정됩니다. 만약 퇴직 후 연금 수급 자격이 유지된다면, 연금은 계속 지급될 수 '
 '있습니다. 그러나 미국으로 이주하는 경우, 연금 수급에 영향을 미칠 수 있는 여러 요인이 있으므로, 관련 법률 및 규정을 확인하고 필요한 '
 '절차를 따르는 것이 중요합니다.')


In [ ]:
i = 0
for item in response.source_nodes:
    i +=1
    print(f"< Source {i} >\n")
    print(item.metadata)
    pprint(item.text)

In [ ]:
from llama_index.core.schema import IndexNode
data_list = ['공무원 성과평가','공무원 징계령','공무원 보수 규정','공무원 수당','공무원 연금','공무원 임용','공직자 윤리']
nodes = [
    IndexNode(text=summary, index_id=f"{data_list[idx]}")
    for idx, summary in enumerate(summaries)
]

In [ ]:
nodes

In [6]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext
import qdrant_client

# Qdrant 클라우드 DB 연결
client = qdrant_client.QdrantClient(
    url="", 
    api_key="",
)
# VectorstoreIndex의 Backend로써 storage_context 부여 및 인덱싱
vector_store = QdrantVectorStore(client=client, collection_name="example")
#storage_context = StorageContext.from_defaults(vector_store=vector_store)
#index = VectorStoreIndex(documents, storage_context=storage_context)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [7]:

from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)

filters_1 = MetadataFilters(
    filters=[
        MetadataFilter(key="file_path", operator=FilterOperator.EQ, value="/Users/doosolini/Documents/GitHub/FastCampus_LlamaIndex/Week 5/data/공무원 성과평가 등에 관한 규정(대통령령)(제33149호)(20221227).pdf"),
    ]
)

filters_2 = MetadataFilters(
    filters=[
        MetadataFilter(key="file_name", operator=FilterOperator.EQ, value="공무원 징계령(대통령령)(제33962호)(20240101).pdf"),
    ]
)

filters_3 = MetadataFilters(
    filters=[
        MetadataFilter(key="file_name", operator=FilterOperator.EQ, value="공무원보수규정(대통령령)(제34099호)(20240112).pdf"),
    ]
)

filters_4 = MetadataFilters(
    filters=[
        MetadataFilter(key="file_name", operator=FilterOperator.EQ, value="공무원수당 등에 관한 규정(대통령령)(제34618호)(20240701).pdf"),
    ]
)

filters_5 = MetadataFilters(
    filters=[
        MetadataFilter(key="file_name", operator=FilterOperator.EQ, value="공무원연금법(법률)(제19513호)(20230630).pdf"),
    ]
)

filters_6 = MetadataFilters(
    filters=[
        MetadataFilter(key="file_name", operator=FilterOperator.EQ, value="공무원임용령(대통령령)(제34608호)(20240627).pdf"),
    ]
)

filters_7 = MetadataFilters(
    filters=[
        MetadataFilter(key="file_name", operator=FilterOperator.EQ, value="공직자윤리법(법률)(제19563호)(20240719).pdf"),
    ]
)

In [8]:
query_engine_1=index.as_query_engine(filters=filters_1)
query_engine_2=index.as_query_engine(filters=filters_2)
query_engine_3=index.as_query_engine(filters=filters_3)
query_engine_4=index.as_query_engine(filters=filters_4)
query_engine_5=index.as_query_engine(filters=filters_5)
query_engine_6=index.as_query_engine(filters=filters_6)
query_engine_7=index.as_query_engine(filters=filters_7)

In [9]:
naive_engine = index.as_query_engine()


In [10]:
from llama_index.core.tools import QueryEngineTool

query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=query_engine_1,
        description=(
            "공무원의 업무 수행에 대한 성과를 평가하는 기준과 절차에 관련된 내용을 물어볼 때 사용"
        )
    ),
    QueryEngineTool.from_defaults(
        query_engine=query_engine_2,
        description=(
            "공무원의 직무 태만이나 법령 위반 등에 대한 징계 절차와 제재 관련된 내용을 물어볼 때 사용"
        )
    ),
    QueryEngineTool.from_defaults(
        query_engine=query_engine_3,
        description=(
            "공무원의 급여, 임금, 보수 관련 내용을 물어볼 때 사용"
        )
    ),
    QueryEngineTool.from_defaults(
        query_engine=query_engine_4,
        description=(
            "공무원이 직무 수행 중 받는 다양한 수당에 관해 물어볼 때 사용"
        )
    ),
    QueryEngineTool.from_defaults(
        query_engine=query_engine_5,
        description=(
            "공무원이 퇴직 후 받는 연금과 관련된 내용을 물어볼 때 사용"
        )
    ),
    QueryEngineTool.from_defaults(
        query_engine=query_engine_6,
        description=(
            "공무원의 채용, 임용 절차, 승진과 특진 및 조건과 관련된 내용을 물어볼 때 사용"
        )
    ),
    QueryEngineTool.from_defaults(
        query_engine=query_engine_7,
        description=(
            "공무원 및 공직자의 윤리적 기준과 행동 강령에 관련된 내용을 물어볼 때 사용"
        )
    ),
]


In [11]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector
)

master_engine = RouterQueryEngine(
    selector=PydanticMultiSelector.from_defaults(),
    query_engine_tools=query_engine_tools
)

In [12]:
response = master_engine.query(
    '공무원으로 일하다가 미국인이 되기로 결심했어. 내 연금은 어떻게 되는거야?'
)

In [13]:
pprint(response.response)

('공무원으로 일하다가 미국인이 되기로 결심한 경우, 연금인 급여를 받을 권리가 있는 사람은 국적을 상실한 달의 다음 달부터 지급되는 연금인 '
 '급여를 일시금으로 받을 수 있습니다. 이 일시금은 국적을 상실한 달의 다음 달을 기준으로 한 4년분의 연금에 상당하는 금액으로 '
 '지급됩니다.')


In [14]:
pprint(str(response.metadata["selector_result"]))

("selections=[SingleSelection(index=4, reason='The question specifically asks "
 "about retirement and pension related to becoming a U.S. citizen.')]")


In [15]:
response.metadata

{'d0d7e3f6-a779-453f-9b94-2babd19f7873': {'page_label': '7',
  'file_name': '공무원연금법(법률)(제19513호)(20230630).pdf',
  'file_path': '/Users/doosolini/Downloads/streamlit_test/data/공무원연금법(법률)(제19513호)(20230630).pdf',
  'file_type': 'application/pdf',
  'file_size': 196708,
  'creation_date': '2024-08-23',
  'last_modified_date': '2024-08-23'},
 'selector_result': MultiSelection(selections=[SingleSelection(index=4, reason='The question specifically asks about retirement and pension related to becoming a U.S. citizen.')])}

# 쿼리라우터 위에 쿼리 디컴포저 추가하기

In [16]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
import asyncio
from tqdm.asyncio import tqdm
from llama_index.core.prompts import PromptTemplate
from typing import Any, Dict, List
query_gen_str = """
너는 사용자가 대충 쓴 질문에 대해서, 최대한 답변하기 위한 근거를 찾기 위한 다수의 서치 쿼리를 생성해 내야해.
사용자의 질문은 기본적으로 공무원 관련된 규정이니까 기본적인 컨텍스트를 공무원으로 인지해.
{num_queries}개의 서치 쿼리를 만들어 내고, 하나당 한줄씩 사용해.
Query: {query}
Queries:
"""
query_gen_prompt = PromptTemplate(query_gen_str)

llm_decomposer = OpenAI(model="gpt-4o-mini")

def generate_queries(llm, query: str, num_queries: int = 3):
    response = llm.predict(
        query_gen_prompt, num_queries=num_queries, query=query
    )
    # assume LLM proper put each query on a newline
    queries = response.split("\n")
    queries_str = "\n".join(queries)
    print(f"Generated queries:\n{queries_str}")
    return queries

def run_queries(queries, engine):
    """Run queries against retrievers."""
    task_results = []
    for query in queries:
        task_results.append(engine.query(query))

    #task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict

def fuse_results(results_dict, similarity_top_k: int = 2):
    """Fuse results."""
    k = 60.0  # `k` is a parameter used to control the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]

class FusionRetriever(BaseRetriever):
    """Ensemble retriever with fusion."""

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = generate_queries(
            self._llm, query_bundle.query_str, num_queries=3
        )
        results = asyncio.run(run_queries(queries, self._retrievers))
        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )

        return final_results

In [17]:
import nest_asyncio
nest_asyncio.apply()

In [18]:
query = '5급 및 7급 공무원(우정직 포함)이 특별승진하려면 어떤 조건이 필요한가요?'
queries = generate_queries(query=query,llm=llm_decomposer)

Generated queries:
1. "5급 7급 공무원 특별승진 조건 우정직"
2. "특별승진 공무원 규정 5급 7급"
3. "우정직 공무원 특별승진 자격 요건"


In [19]:
result_dict = run_queries(queries, master_engine)

In [20]:
result_dict

{('1. "5급 7급 공무원 특별승진 조건 우정직"',
  0): Response(response='5급 및 7급 이하 공무원, 특히 우정직 공무원의 경우 특별승진임용을 받을 수 있는 조건은 다음과 같습니다. 해당 공무원은 승진소요최저연수에 도달해야 하며, 특별승진임용 시 승진후보자 명부의 순위에 관계없이 승진 심사를 거쳐 바로 상위 직급으로 승진임용될 수 있습니다. 우정직공무원의 경우에는 우정4급 이하 공무원이 해당됩니다.', source_nodes=[NodeWithScore(node=TextNode(id_='a4d3c0c8-2706-49c4-a943-7503ea399889', embedding=None, metadata={'page_label': '22', 'file_name': '공무원임용령(대통령령)(제34608호)(20240627).pdf', 'file_path': '/Users/doosolini/Downloads/streamlit_test/data/공무원임용령(대통령령)(제34608호)(20240627).pdf', 'file_type': 'application/pdf', 'file_size': 268393, 'creation_date': '2024-08-23', 'last_modified_date': '2024-08-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='법제처                                                            22    

In [21]:
inter_queries = []

for k,v in result_dict.items():
    inter_queries.append(v.response)

In [22]:
inter_queries

['5급 및 7급 이하 공무원, 특히 우정직 공무원의 경우 특별승진임용을 받을 수 있는 조건은 다음과 같습니다. 해당 공무원은 승진소요최저연수에 도달해야 하며, 특별승진임용 시 승진후보자 명부의 순위에 관계없이 승진 심사를 거쳐 바로 상위 직급으로 승진임용될 수 있습니다. 우정직공무원의 경우에는 우정4급 이하 공무원이 해당됩니다.',
 '특별승진 공무원 규정에 따르면, 5급 및 7급 이하 공무원은 승진후보자 명부의 순위에 관계없이 승진 심사를 거쳐 바로 상위 직급으로 승진임용될 수 있습니다. 이는 특별승진임용의 일환으로, 특정 요건을 충족하는 공무원에게 적용됩니다.',
 '우정직 공무원의 특별승진 자격 요건은 다음과 같습니다:\n\n1. 승진소요최저연수에 도달한 공무원이어야 하며, 특정 경우에는 승진소요최저연수를 6개월 단축할 수 있습니다.\n2. 재직기간 중 중징계 처분이나 특정 사유로 경징계 처분을 받지 않아야 하며, 명예퇴직일 전날까지 해당 계급에서 1년 이상 재직해야 합니다.\n3. 특별승진임용 시 승진후보자 명부의 순위에 관계없이 승진 심사를 거쳐 바로 상위 직급으로 승진임용될 수 있습니다. \n\n이 외에도 특별한 공적이 있는 경우에는 정원을 초과하여 임용할 수 있으며, 심사 절차는 인사혁신처장이 정합니다.']

In [23]:
query_fuse_str = """
너는 공무원 관련 규정을 묻는 사용자에게 답을 해주는 봇이야.
사용자의 질문에 답을 잘 해주기 위해, 미리 사용자의 질문을 분해해서 각각의 분해된 중간 답안을 도출해 놓은 상태야.
사용자의 질문을 도출된 중간 답안들만을 이용해서 정확하고 구체적인 법적 규정이 포함되어 있는 최종 답변을 생성해.

중간 답안1: {answer1}
중간 답안2: {answer2}
중간 답안3: {answer3}

사용자 질문: {query}
최종 답변:
"""
query_fuse_prompt = PromptTemplate(query_fuse_str)
llm= OpenAI(model="gpt-4o-mini",temperature=0)
inter_queries = []

for k,v in result_dict.items():
    inter_queries.append(v.response)
final_response = llm.predict(
        query_fuse_prompt, answer1=inter_queries[0], answer2=inter_queries[1], answer3=inter_queries[2], query=query
    )

In [24]:
pprint(final_response)

('5급 및 7급 이하 공무원, 특히 우정직 공무원이 특별승진임용을 받기 위해서는 다음과 같은 조건을 충족해야 합니다:\n'
 '\n'
 '1. **승진소요최저연수 도달**: 해당 공무원은 승진소요최저연수에 도달해야 하며, 특정 경우에는 이 연수를 6개월 단축할 수 '
 '있습니다.\n'
 '\n'
 '2. **징계 처분 여부**: 재직기간 중 중징계 처분이나 특정 사유로 경징계 처분을 받지 않아야 하며, 명예퇴직일 전날까지 해당 '
 '계급에서 1년 이상 재직해야 합니다.\n'
 '\n'
 '3. **승진 심사**: 특별승진임용 시 승진후보자 명부의 순위에 관계없이 승진 심사를 거쳐 바로 상위 직급으로 승진임용될 수 '
 '있습니다.\n'
 '\n'
 '4. **특별한 공적**: 특별한 공적이 있는 경우에는 정원을 초과하여 임용할 수 있습니다.\n'
 '\n'
 '이러한 조건을 충족하는 경우, 우정직 공무원은 특별승진임용을 통해 상위 직급으로의 승진이 가능하게 됩니다. 심사 절차는 인사혁신처장이 '
 '정합니다.')


In [25]:
query = '고위공무원에게 보직 없이 근무할 경우 연봉은 어떻게 지급되나요?'
queries = generate_queries(query=query,llm=llm_decomposer)

result_dict = run_queries(queries, master_engine)
inter_queries = []

for k,v in result_dict.items():
    inter_queries.append(v.response)
final_response = llm.predict(
        query_fuse_prompt, answer1=inter_queries[0], answer2=inter_queries[1], answer3=inter_queries[2], query=query
    )
pprint(final_response)

Generated queries:
1. "고위공무원 보직 없이 근무 시 연봉 지급 규정"
2. "고위공무원 연봉 산정 기준 및 보직 없는 경우"
3. "공무원 보직 없는 근무 시 급여 지급 방식"
('고위공무원이 보직 없이 근무할 경우 연봉 지급 기준에 대한 구체적인 정보는 제공되지 않았습니다. 이러한 경우의 연봉 계산 기준이나 보직이 '
 '없는 상황에 대한 규정은 관련 법령이나 공식 문서를 참조하는 것이 좋습니다. 고위공무원의 보수 및 직무 배치에 관한 구체적인 지침을 '
 '확인하시기 바랍니다.')
